In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

/Users/matsumotoyuka/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/matsumotoyuka/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [8]:
excel_file_path = 'all_data.xlsx'
csv_file_path = 'all_data.csv'
df = pd.read_excel(excel_file_path, sheet_name=0)
# 削除する列のリスト
columns_to_drop = ['Density [kg/m^3]', '緯度', '経度', 'DO eq', '備考', '水深', 'pH', 'Pressure', 'O2 weiss 1970','DIC[μmol/L@25℃]','Depth[m]', 'DO[ µmol/kg]', 'TA [μmol/kg]','Salinity']
df.drop(columns=columns_to_drop, inplace=True)
# CSVファイルとして保存
df.to_csv(csv_file_path, index=False)
print(f"CSVファイルが保存されました: {csv_file_path}")
df

CSVファイルが保存されました: all_data.csv


,No.,pCO2 [μatm],AOU,excess DIC,T,nDIC+0.768DO,DIC-0.5TA+0.83DO,場所,日時,DO Saturation,label
0,1,287.110917,-59.375370,-76.860917,27.8250,2114.488440,944.092151,橘湾,2020-08-31,1.295823,0.0
1,2,425.777660,12.204354,2.096881,25.6610,2166.235913,985.000393,橘湾,2020-08-31,0.941249,0.0
2,3,420.944503,14.201825,-0.024379,25.3680,2164.846536,984.128603,橘湾,2020-08-31,0.931941,0.0
3,4,487.481913,72.631801,26.032198,22.6560,2172.105752,989.467402,橘湾,2020-08-31,0.666868,0.0
4,5,466.192694,23.793279,18.832014,25.5250,2175.477671,997.066525,橘湾,2020-08-31,0.885662,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1894,1895,542.771000,118.587515,49.811000,10.6016,2265.132400,1143.534000,北大西洋,NaT,0.521004,NaN
1895,1896,557.977000,130.831224,53.484000,10.1420,2265.232200,1141.832000,北大西洋,NaT,0.483192,NaN
1896,1897,519.493000,119.491733,43.128000,10.7688,2255.845200,1134.107000,北大西洋,NaT,0.516312,NaN
1897,1898,538.981000,128.104189,48.153000,10.0288,2263.360400,1140.654000,北大西洋,NaT,0.493668,NaN


In [13]:
def analyze_with_prediction_interval(df, param1, param2, figsize=(12, 8)):
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所']=='苫小牧'].copy()
    #データを準備
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(x.flatten(), y)

    # 回帰直線の予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    y_pred = slope * x_line.flatten() + intercept

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("whitegrid")

    # 実データのプロット
    data_point = df[df['場所'].isin(['オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    plt.scatter(data_point[param2],
               data_point[param1],
               color='black',
               marker='o',
               alpha=1,
               s=50)

    # 回帰直線のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Regression Line (y = {slope:.3f}x + {intercept:.3f})')

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=12)
    plt.ylabel(f'{param1}', fontsize=12)
    plt.title(f'Data Distribution in NOAA with Regression Line: {param1} vs {param2}', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.savefig(f'regression_analysis_NOAA_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 全データポイントについて、回帰線からの予測値を計算
    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = slope * x_all.flatten() + intercept

    # 実際の値と予測値を比較し、回帰線より下のポイントを特定
    actual_y = df[param1].values
    outliers = df[actual_y < y_pred_data].copy()
    outliers_seepage = outliers[outliers['場所'].isin(['オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])]['No.'].tolist()
    print(f"  - 回帰線より下にあるデータ: No.{outliers_seepage}")

    return outliers

In [14]:
def analyze_with_prediction_interval_pCO2_DOsaturation(df, param1, param2, figsize=(12, 8)):
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所']=='苫小牧'].copy()
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 対数変換
    log_x = np.log(x)
    log_y = np.log(y)

    # 対数空間で線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_x.flatten(), log_y)

    # 累乗関数のパラメータに変換
    a = np.exp(intercept)  # 係数
    b = slope  # 指数

    # 予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    # 累乗関数での予測値
    y_pred = a * (x_line ** b).flatten()

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("whitegrid")

    # 実データのプロット
    data_point = df[df['場所'].isin(['オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    plt.scatter(data_point[param2],
               data_point[param1],
               color='black',
               marker='o',
               alpha=1,
               s=50)

    # 累乗回帰曲線のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Power Regression (y = {a:.2f}x^{b:.2f})')

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=12)
    plt.ylabel(f'{param1}', fontsize=12)
    plt.title(f'Data Distribution in NOAA with Power Regression: {param1} vs {param2}', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.savefig(f'power_regression_analysis_NOAA_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 全データポイントについて、累乗回帰での予測値を計算
    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = a * (x_all ** b).flatten()

    # 実際の値と予測値を比較し、回帰線より下のポイントを特定
    actual_y = df[param1].values
    outliers = df[actual_y < y_pred_data].copy()  # 回帰線より下のデータを抽出
    outliers_seepage = outliers[outliers['場所'].isin(['オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])]['No.'].tolist()
    print(f"  - 回帰線より下にあるデータ: No.{outliers_seepage}")

    return outliers

In [15]:
outliers1 = analyze_with_prediction_interval_pCO2_DOsaturation(df, 'pCO2 [μatm]', 'DO Saturation')
outliers2 = analyze_with_prediction_interval(df, 'excess DIC', 'AOU')
outliers3 = analyze_with_prediction_interval(df, 'nDIC+0.768DO', 'T')
outliers4 = analyze_with_prediction_interval(df, 'DIC-0.5TA+0.83DO', 'T')

  - 回帰線より下にあるデータ: No.[407, 411, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 490, 516, 518, 520, 521, 524, 526, 527, 528, 533, 536, 540, 542, 546, 548, 549, 551, 558, 561, 567, 568, 569, 571, 572, 573, 574, 575, 577, 579, 580, 582, 583, 584, 586, 587, 590, 592, 593, 594, 595, 597, 598, 599, 600, 601, 606, 607, 608, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703